In [1]:
# game room

broken_shackles = {
    "name": "broken shackles",
    "type": "furniture",
}

dungeon_door = {
    "name": "dungeon door",
    "type": "door",
}

finger_bone = {
    "name": "finger bone",
    "type": "key",
    "target": dungeon_door,
}

corpse = {
    "name": "corpse",
    "type": "furniture",
}

dungeon = {
    "name": "dungeon",
    "type": "room",
}

outside = {
  "name": "outside"
}

# hallway

torture_table = {
    "name": "torture table",
    "type": "furniture",
}

weird_stone_on_the_wall = {
    "name": "weird stone on the wall",
    "type": "door",
}

sledgehammer = {
    "name": "sledgehammer",
    "type": "key",
    "target": weird_stone_on_the_wall,
}

reinforced_steel_door = {
    "name": "reinforced steel door",
    "type": "door",
}

hallway = {
    "name": "hallway",
    "type": "room",
}

secret_chamber = {
    "name": "secret chamber",
    "type": "room",
}

outside = {
  "name": "outside"
}

# secret chamber

crumbled_bed = {
    "name": "crumbled bed",
    "type": "furniture",
}

weird_stone_on_the_wall = {
    "name": "weird stone on the wall",
    "type": "door",
}

big_key = {
    "name": "big key",
    "type": "key",
    "target": reinforced_steel_door,
}

magic_portal = {
    "name": "magic portal",
    "type": "door",
}

magic_pendant = {
    "name": "magic pendant",
    "type": "key",
    "target": magic_portal,
}

glass_cabinet = {
    "name": "glass cabinet",
    "type": "furniture",
}

hallway = {
    "name": "hallway",
    "type": "room",
}

outside = {
  "name": "outside"
}

wormhole = {
    "name": "wormhole",
    "type": "exit",
    "description": "This strange phenomenon is out of this world. However, it seems to be locked with a spell.",
    "code": "leomunds opulent outhouse"  
}

# morganas_room

altar = {
    "name": "altar",
    "type": "furniture",
}

reinforced_steel_door = {
    "name": "reinforced steel door",
    "type": "door",
}

hallway = {
    "name": "hallway",
    "type": "room",
}

morganas_room = {
    "name": "morganas room",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [dungeon, hallway, secret_chamber, morganas_room, outside]

all_doors = [dungeon_door, weird_stone_on_the_wall, reinforced_steel_door, magic_portal, wormhole]

# define which items/rooms are related

object_relations = {
    "dungeon": [broken_shackles, corpse, dungeon_door],
    "hallway": [torture_table, dungeon_door, weird_stone_on_the_wall, reinforced_steel_door],
    "secret chamber": [glass_cabinet, weird_stone_on_the_wall, crumbled_bed, wormhole],
    "morganas room": [altar, magic_portal],
    "corpse": [finger_bone],
    "torture table": [sledgehammer],
    "crumbled bed": [big_key],
    "glass cabinet": [magic_pendant],
    "outside": [magic_portal],
    "magic portal": [outside],
    "dungeon door": [hallway, dungeon],
    "weird stone on the wall": [secret_chamber, hallway],
    "reinforced steel door": [morganas_room, hallway]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": dungeon,
    "keys_collected": [],
    "target_room": outside
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up trapped inside a dungeon. You don't remember why you are here and what had happened before. But you do see a powerful dark witch leaving as she vanishes into the walls. As you listen closely you hear her say: leomunds opulent outhouse! Suddenly you hear a loud noise which sounds like an evil laugh. You don't have much time to attempt your escape before she returns!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
                    
                    
            # We added this line of code to trigger an alternative ending. It is written here how the wormhole triggers the end of the game and the conditions it needs to meet
            elif item["name"] == "wormhole": 
                output += item["description"]
                if input("Would you like to enter the wormhole? Enter 'yes' or 'no': ").strip().lower() == 'yes':
                    user_code = input("Speak the spell: ").strip().lower()
                    if user_code == item["code"]:
                        print("You have successfully reached Nirvana. Congratulations! You have escaped from Morgana's grasp!")
                        exit()  
                        return
                    else:
                        print("The wormhole seems to reject you.")
                    
                    
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
    
    if(next_room and input("Do you want to go to the next room? Ener 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up trapped inside a dungeon. You don't remember why you are here and what had happened before. But you do see a powerful dark witch leaving as she vanishes into the walls. As you listen closely you hear her say: leomunds opulent outhouse! Suddenly you hear a loud noise which sounds like an evil laugh. You don't have much time to attempt your escape before she returns!
You are now in dungeon
